# Filtrado de publicaciones en el contexto deseado

En este cuaderno se hace uso de los servicios de Azure OpenAI para filtrar publicaciones (de Twitter, 4chan, etc.) en las que se tiene una palabra de interés, pero que se emplea en un contexto diferente al requerido.

Se especificará una lista de palabras/frases objetivo y, mediante el uso de la API de Azure OpenAI, se determinará si la publicación contiene esta palabra y es usada en el contexto deseado.

En este cuaderno se usará el lenguaje de Python y se hará uso de la biblioteca [OpenAI](https://pypi.org/project/openai/).

Para ejecutar este cuaderno, mira los requisitos en el archivo [README.md](./README.md) y selecciona el entorno virtual creado para este proyecto.

## Palabra: `colgarme`

Para el caso de las publicaciones con la palabra "colgarme", se tienen casos en el que la palabra es usada con intenciones autolesivas, como los siguientes:

* <blockquote> OAA me hace dar ganas de colgarme, uhhh si. https://t.co/3vc7ELNvtH </blockquote>
* <blockquote> *procedo a colgarme del abanico* </blockquote>
* <blockquote> No me siento preparada para colgarme hoy :-) </blockquote>
* <blockquote> Tengo más ganas de colgarme de los huevos en un anzuelo para pescar tiburones que de seguir cursando esta materia </blockquote>
* <blockquote> Pero antes de colgarme debo oir el álbum de Taylor por qué la nena anda al tope </blockquote>
* <blockquote> Ese filtro me quitó las ganas de colgarme 💗. https://t.co/LtvKc0cKsI </blockquote>

No obstante, el propósito de la palabra en una oración no siempre es el mismo.

<blockquote>
* Por colgarme mirando Superman me olvidé de cargar el celular y estoy llegando tarde al colegio :(

    -> Donde "colgarme" es usado como "distraerme"
</blockquote>

<blockquote>
* CHICAS YO BAILO POLE DANCE NATURALMENTE SIN ACADEMIAS PORQUE FROTARME Y COLGARME DEL CAÑO ES MI TALENTO NATO

    -> Donde "colgarme" es usado como "sujetarme"

    * Nótese que hay objetos de los que puede sujetarse, como un árbol, pero que también es candidato a ser usadopara colgarse del cuello. Todas aquellas entidades de las que se pueda inferir un potencial uso nocivo se considerarán como alarmantes.
</blockquote>

<blockquote>
* Una mina con quien colgarme toda la noche escuchando rock

    -> Donde "colgarme" es usado como "pasar tiempo"
</blockquote>

<blockquote>
* espero no colgarme e ir al colegio hoy

    -> Donde "colgarme" es usado como "pasarme de largo"
</blockquote>

etc...

Se espera únicamente conservar los tweets englobados en la primera categoría.

Del modelo de completición de OpenAI, se espera inferir el contexto de la palabra y que este devuelva un 0, si es
que el contexto no es uno alarmante, o un 1, en caso contrario.

Primero que nada, hagamos una función genérica que nos permita hacer uso de la API de OpenAI para inferir el contexto de una palabra.

In [3]:
# Importar las librerías necesarias
import os
import openai

In [6]:
def obtener_respuesta(mensaje : str, nombre_modelo : str = "gpt-35-turbo-deployed"):

    """
        Función que obtiene la respuesta del asistente virtual.
    """

    respuesta = openai.ChatCompletion.create(
            engine=nombre_modelo,
            messages=mensaje,
            temperature=0.2,
            max_tokens=5,
            top_p=1.0,
            frequency_penalty=1.0,
            presence_penalty=0.0,
            stop=None
        )

    
    return respuesta['choices'][0]['message']['content']

In [23]:
def determinar_correspondencias_con_contexto(palabra_frase : str, textos: list[str], intencion : str):

    """
        Función que determina si la respuesta del asistente virtual corresponde con el contexto de la pregunta.

        Parámetros:
            - palabra_frase: palabra o frase que se quiere determinar si corresponde con el contexto.
            - texto: texto en el que se encuentra la palabra o frase.
            - intención: intención con la que se espera que la palabra sea usada.

        Retorno:
            - 0 si la intención de la palabra en el texto proporcionado no corresponde con la intención esperada.
            - 1 en caso contrario.
    """

    textos_para_mensaje = ""

    for indice, texto in enumerate(textos):
        textos_para_mensaje += f"Texto # {indice + 1}: {texto}\n"

    mensaje = [
        {
        "role": "system",
        "content": f"""
                Eres un asistente que determina si la palabra o frase '{palabra_frase}',
                dentro de los siguientes textos cumple con una intención dada.\n
                Aquí tienes los textos:\n
                {textos_para_mensaje}\n
                A continuación se te proporciona detalles sobre el cumplimiento:\n 
                {intencion}.\n

                Responde únicamente con un 1 si la palabra o frase cumple con la intención dada,
                en caso contrario responde con un 0.\n

                La respuesta deberá ser algo como '0,1,0,1', según el número de textos dados.\n
            """
        },
    ]

    respuesta = obtener_respuesta(mensaje)

    # Obtenemos los valores de la respuesta
    respuestas = respuesta.split(",")
    # Devolvemos una lista de booleanos
    return [True if int(respuesta) == 1 else False for respuesta in respuestas]

Ya podemos trabajar con nuestra función genérica para inferir el contexto de una palabra.

No obstante, es necesario configurar las credenciales de OpenAI para poder hacer uso de la API.

In [8]:
# Traer las credenciales de OpenAI
from dotenv import load_dotenv
load_dotenv()
dotenv_path = os.path.join(os.path.abspath(""), '.env')

# Configurar la API de OpenAI
openai.api_type = "azure"
openai.api_base = os.getenv("OPENAI_API_ENDPOINT")
openai.api_version = "2023-03-15-preview"
openai.api_key = os.getenv("OPENAI_API_KEY")

In [25]:
# Realizar una prueba

palabra_frase = "fresa"
textos = [
    "La fresa es una fruta deliciosa",
    "La fresa es una pieza de metal que se usa para cortar metal",
]
# intencion = "La palabra fresa se usa para referirse a una máquina que se usa para cortar metal"
intencion = "La palabra fresa se usa para referirse a una fruta"

cumplimientos = determinar_correspondencias_con_contexto(palabra_frase, textos, intencion)

for indice, cumple in enumerate(cumplimientos):
    cumple_str = "cumple" if cumple else "no cumple"
    texto = textos[indice]
    print(
        """
            La palabra o frase '{}' en el texto '{}'
            {}
            cumple con la intención.
        """.format(palabra_frase, texto, cumple_str)
    )



            La palabra o frase 'fresa' en el texto 'La fresa es una fruta deliciosa'
            cumple
            cumple con la intención.
        

            La palabra o frase 'fresa' en el texto 'La fresa esu una pieza de metal que se usa para cortar metal'
            no cumple
            cumple con la intención.
        
